<a href="https://colab.research.google.com/github/SatoJin02/BDA_course25/blob/main/ex02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 駅データの可視化（Folium & Plotly Express）

このノートブックでは、駅データ（緯度・経度を含む CSV ファイル）を読み込み、  
**Folium** と **Plotly Express** を用いて地図上に可視化します。  


## 1. 必要なライブラリのインポート

- pandas : データフレーム操作  
- folium : Leaflet.js ベースの地図描画  
- plotly.express : Mapbox ベースのインタラクティブ描画


In [ ]:
import pandas as pd
import folium
import plotly.express as px

## 2. データの読み込みと前処理

CSVファイル `station_Info.csv` を読み込みます。  
`Location` カラムには `"Point(経度 緯度)"` という形式で文字列が入っているため、  
正規表現で経度（Longitude）と緯度（Latitude）を抽出して数値化します。


In [ ]:
# CSV読み込み
df = pd.read_csv("/content/station_Info.csv")

# 緯度経度を抽出
df[["Longitude", "Latitude"]] = df["Location"].str.extract(
    r'Point\(([-0-9.]+) ([-0-9.]+)\)'
).astype(float)

## 3. Folium を使った地図表示

- 地図の中心をデータの平均座標に設定  
- 各駅にマーカーを追加  
- マーカーをクリックすると住所が表示され、ホバーで stationId を確認できます  
- `simple_dot_plot.html` として保存して、外部でも開けるようにしています  

In [ ]:
# ==============================
# Folium
# ==============================
this_map = folium.Map(
    location=[df["Latitude"].mean(), df["Longitude"].mean()],
    zoom_start=10,
    prefer_canvas=True
)

# マーカー追加
for _, row in df.iterrows():
    folium.Marker(
        location=[row["Latitude"], row["Longitude"]],
        popup=row["Address"],
        tooltip=row["stationId"]
    ).add_to(this_map)

# 範囲をデータにフィット
this_map.fit_bounds(this_map.get_bounds())

# HTMLに保存
this_map.save("simple_dot_plot.html")

# Colab上でのインライン表示
this_map


## 4. Plotly Express を使った地図表示

- Plotly Express の `scatter_mapbox` を利用  
- 各駅を `stationId` ごとに色分けして表示  
- ホバーすると stationId と住所が表示される  


In [ ]:
# ==============================
# Plotly Express 可視化
# ==============================
fig = px.scatter_mapbox(
    data_frame=df,
    lat="Latitude",
    lon="Longitude",
    hover_name="stationId",
    hover_data=["Address"],
    color="stationId",   # IDごとに色分け
    size_max=15,
    opacity=0.6,
    center={'lat':34.686567, 'lon':135.52000},
    zoom=4,
    height=600,
    width=800
)

# OpenStreetMapスタイルで描画
fig.update_layout(
    mapbox_style="open-street-map",
    margin={"r":0, "t":30, "l":0, "b":0},
    title_text="Station Locations"
)
fig.show()